In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_cluster=pd.read_csv('df_cluster.csv')
df_cluster.head()

,age,gender,security_no,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_session_duration,avg_transaction_value,...,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website,Churn_probability,transaction/day,CLV,recency
0,0.148148,0,XW0DQ7H,2017-08-17,0,xxxxxxxx,2025-08-19 16:08:02,0.991220,0.514859,0.526717,...,0,1,0,0,0,0,0.003610,0.850001,10.591081,0.975000
1,0.407407,0,5K0N3X1,2017-08-28,0,CID21329,2025-08-19 12:38:13,0.990244,0.515803,0.121456,...,0,0,1,0,0,0,0.003547,0.217005,8.883273,0.987342
2,0.629630,0,1F2TCL3,2016-11-11,1,CID12313,2025-08-19 22:53:21,0.988293,0.550486,0.204074,...,1,0,0,0,0,0,0.964750,0.308034,0.000148,0.829787
3,0.500000,1,VJGJ33N,2016-10-29,1,CID3793,2025-08-19 15:57:50,0.985366,0.473971,0.246577,...,1,0,0,0,0,0,0.970951,0.469295,0.000066,0.795918
4,0.388889,0,SVZXCWB,2017-09-12,0,xxxxxxxx,2025-08-19 15:46:44,0.994146,0.483866,0.238950,...,1,0,0,0,0,0,0.959872,0.391017,0.000203,0.975000


In [4]:
from sklearn.cluster import KMeans
x_cluster=df_cluster[['CLV','Churn_probability','avg_transaction_value','api_calls_90d']]